# Flask App.py

In [1]:
pip install flask

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from flask import Flask, render_template, request
import joblib
import pandas as pd
import plotly.express as px

# Load the trained model and scaler
model = joblib.load('World_Population_Analysis.pkl')  # Update with your model filename
scaler = joblib.load('scaler.pkl')  # Update with your scaler filename

# Load dataset for related countries and visualization
data = pd.read_csv('world_population.csv')

# Function to predict population based on input features
def predict_population(features):
    features_scaled = scaler.transform([features])  # Scale input features
    return model.predict(features_scaled)[0]  # Predict population

app = Flask(__name__)

# Route for the homepage
@app.route('/')
def index():
    return render_template('index.html')

# Route for handling the form submission
@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        try:
            # Extract input data from the form
            growth_rate = float(request.form['growth_rate'])
            density = float(request.form['density'])
            avg_population = float(request.form['avg_population'])
            area = float(request.form['area'])

            # Prepare the features for prediction
            features = [growth_rate, density, avg_population, area]
            
            # Predict population
            prediction = predict_population(features)

            # Find related countries with similar populations
            matching_countries = data[
                abs(data['2022 Population'] - prediction) < 50000000  # ±50M margin
            ][['Country/Territory', '2022 Population']].head(50)

            # Generate Pie Chart
            pie_fig = px.pie(matching_countries, names='Country/Territory', values='2022 Population',
                              title="Population Distribution of Related Countries")
            pie_chart_html = pie_fig.to_html(full_html=False)

            # Generate Bar Chart
            bar_fig = px.bar(matching_countries, x='Country/Territory', y='2022 Population',
                              title="Comparison of Related Country Populations")
            bar_chart_html = bar_fig.to_html(full_html=False)

            # Render result page with added charts
            return render_template('result.html', 
                                   prediction=int(prediction), 
                                   countries=matching_countries.to_dict(orient='records'),
                                   pie_chart=pie_chart_html,
                                   bar_chart=bar_chart_html)
        except Exception as e:
            return f"Error occurred: {e}"

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Feb/2025 21:05:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2025 21:05:57] "GET /favicon.ico HTTP/1.1" 404 -
C:\Users\91834\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [05/Feb/2025 21:06:20] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2025 21:06:28] "GET / HTTP/1.1" 200 -
C:\Users\91834\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

127.0.0.1 - - [05/Feb/2025 21:06:54] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2025 21:06:58] "GET / HTTP/1.1" 200 -
C:\Users\91834\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but St